In [1]:
import requests 

from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

from time import sleep
import pandas as pd
import numpy as np

# Para visualizar el progreso:
import tqdm

True

# FUNCIONES SCRAPING:

- URLS

In [2]:
def iniciar_scraper(user_agent, scrape_urls):
    
    # Options:
    opts = Options()
    opts.add_argument("--disable-extensions")
    opts.add_argument(f"User-Agent={user_agent}")
    
    # Initialize:
    browser = webdriver.Chrome(options=opts)
    browser.maximize_window()
    sleep(5)
    
    urls_paginas_perfumes = []
    
    inicio = 1
    for url in scrape_urls:

        browser.get(url)
        sleep(5)###########################################################################
        
        #Acepto cookies:
        if inicio == 1:            
            browser.find_element(By.CLASS_NAME, "uc-list-button__accept-all").click()
            sleep(6)########################################################################

        soup = BeautifulSoup(browser.page_source, "html.parser")
        
        num_paginas = int(soup.find("div", class_= "pagination-title pagination-title--with-dropdown").text.split()[-1])        

        urls_paginas = [f"{url}?page={num_pagina}" for num_pagina in range(1,num_paginas)]
        
        urls_paginas_perfumes.extend(urls_paginas)
        
        inicio += 1                
        
    ############################################################################################################    
    browser.close()    
    return urls_paginas_perfumes


- FUNCIÓN INFO:

In [1]:
def scrape_info(url):
    
    response = requests.get(url)
    sleep(10)
    soup = BeautifulSoup(response.text, "html.parser")
    
    productos = soup.find_all("div", class_= "product-grid-column col-sm-6 col-md-4 col-lg-3")
    
    cat = soup.find("h1", class_= "headline-bold headline-bold--small product-overview__headline").text
    
    info_productos = []
    
    for producto in productos:
        
        categoria = cat
        
        try:
            marca = producto.find("div", class_= "text top-brand").text
        except:
            marca = "Sin marca"
            
        try:
            nombre = producto.find("div", class_= "text brand-line").text
        except:
            nombre = "Sin nombre"
        
        try:
            tipo = producto.find("div", class_= "text category").text.strip()
        except:
            tipo = "Sin categoria"           
            
            
        precio = producto.find("div", class_= "price-row").find_all("span", class_= "product-price__price")
        
        if len(precio) == 2:
            try:
                precio_sin_desc = precio[0].text

                precio_con_desc = precio[1].text
            except:
                precio_sin_desc = np.nan

                precio_con_desc = np.nan
                
        if  len(precio) == 1:
            try:
                precio_sin_desc = precio[0].text

                precio_con_desc = "Sin descuento"
            except:
                precio_sin_desc = np.nan

                precio_con_desc = np.nan
                
        descuento = "No aparece"
        
        try:
            evaluacion = producto.find("div", class_= "Xjks1stJBh1GPUKjmb16").find("span", class_= "k3DCj3KuQfDwBmZAVbJN").text.replace(")","").split("\xa0(")
            
            valoracion = evaluacion[0]        
            num_valoraciones = evaluacion[1]
        except:
            valoracion = np.nan
            num_valoraciones = np.nan
        
        
            
        info_productos.append([categoria, marca, nombre, tipo, precio_sin_desc, descuento, precio_con_desc, valoracion, num_valoraciones])
    
    return info_productos

In [68]:
######################################################################################################################################################

In [69]:
######################################################################################################################################################

# Scrapeo URLS

In [21]:
# PONER EL USER-AGENT Y LOS ELEMENTOS DE BUSQUEDA:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"

perfumes_hombre = "https://www.douglas.es/es/c/perfumes/fragancias-de-hombre/0102"
perfumes_mujer = "https://www.douglas.es/es/c/perfumes/fragancias-de-mujer/0101"

busqueda = [perfumes_hombre, perfumes_mujer]

In [22]:
# Inicio el scrapeo de las url's, guardándolas en la siguiente variable

urls_totales = iniciar_scraper(user_agent, busqueda)

C:\Users\regue\AppData\Local\Temp\ipykernel_4384\2451663283.py:9: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(executable_path = path_driver, chrome_options=opts)


In [23]:
df_urls_perfumes = pd.DataFrame(urls_totales, columns= ["url"])

## CSV

In [24]:
# df_urls_perfumes.to_csv("urls_perfumes_douglas.csv", index="False", sep= ",")

In [ ]:
######################################################################################################################################################

In [ ]:
######################################################################################################################################################

# Scrapeo la info de todos los productos de cada página:

In [63]:
df_urls_perfumes = pd.read_csv("urls_perfumes_douglas.csv")

In [64]:
info_productos = []

for url in tqdm.tqdm(df_urls_perfumes["url"].values):

    info = scrape_info(url)
    info_productos.extend(info)

100%|██████████| 74/74 [13:48<00:00, 11.19s/it]


In [65]:
df_info_productos = pd.DataFrame(info_productos, columns= ["categoria", "marca", "nombre", "tipo", "precio_sin_desc", "descuento", "precio_con_desc", "valoracion", "num_valoraciones"] )

## CSV

In [66]:
# df_info_productos.to_csv("datos_perfumes_douglas.csv", index="False", sep= ",")

In [ ]:
######################################################################################################################################################

In [ ]:
######################################################################################################################################################